In [1]:
spark.sql("SET spark.sql.parquet.int96TimestampConversion=true")
spark.conf.get("spark.sql.parquet.int96TimestampConversion")

Out[206]: 'true'

In [2]:
"SET spark.sql.parquet.int96TimestampConversion=true"

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-2095125481362399> in <module> 
 ----> 1 spark . sql . parquet . int96TimestampConversion = true

 NameError : name 'true' is not defined

In [3]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.types import *
from pyspark.storagelevel import StorageLevel
from pyspark.sql.functions import udf

In [4]:

list=[[1,'event1',10,'2016-05-01'],
      [2,'event2',100,'2016-05-01'],
      [1,'event3',20,'2016-05-02'],
      [1,'event1',15,'2016-05-03'],
      [3,'event1',13,'2016-05-04'],
      [1,'event2',12,'2016-05-04'],
      [1,'event3',11,'2016-05-05']]
df= spark.createDataFrame(list,['KEY','Event_Type','metric','Date'])
df.show()

+---+----------+------+----------+
KEY|Event_Type|metric| Date|
+---+----------+------+----------+
 1| event1| 10|2016-05-01|
 2| event2| 100|2016-05-01|
 1| event3| 20|2016-05-02|
 1| event1| 15|2016-05-03|
 3| event1| 13|2016-05-04|
 1| event2| 12|2016-05-04|
 1| event3| 11|2016-05-05|
+---+----------+------+----------+

In [5]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

df1= df.withColumn("Date", F.col("Date").cast("timestamp")).persist(StorageLevel.MEMORY_AND_DISK)

In [6]:
days = lambda i: i * 86400
w1 = Window().partitionBy(F.col("KEY")).orderBy(F.col("Date").cast('long')).rangeBetween(-days(29), 0)
w2 = Window().partitionBy(F.col("KEY")).orderBy(F.col("Date").cast('long')).rangeBetween(-days(59), 0)
w3 = Window().partitionBy(F.col("KEY")).orderBy(F.col("Date").cast('long')).rangeBetween(-days(89), 0)

df2=df1.withColumn("last30days_stdev", F.stddev("metric").over(w1))\
       .withColumn("last60days_stdev", F.stddev("metric").over(w2))\
       .withColumn("last90days_stdev", F.stddev("metric").over(w3))\
       .orderBy(df1.Date.desc())                                          

In [7]:
df2.show()

+---+----------+------+-------------------+------------------+------------------+------------------+
KEY|Event_Type|metric| Date| last30days_stdev| last60days_stdev| last90days_stdev|
+---+----------+------+-------------------+------------------+------------------+------------------+
 1| event3| 11|2016-05-05 00:00:00| 4.03732584763727| 4.03732584763727| 4.03732584763727|
 3| event1| 13|2016-05-04 00:00:00| NaN| NaN| NaN|
 1| event2| 12|2016-05-04 00:00:00| 4.349329450233296| 4.349329450233296| 4.349329450233296|
 1| event1| 15|2016-05-03 00:00:00| 5.0| 5.0| 5.0|
 1| event3| 20|2016-05-02 00:00:00|7.0710678118654755|7.0710678118654755|7.0710678118654755|
 1| event1| 10|2016-05-01 00:00:00| NaN| NaN| NaN|
 2| event2| 100|2016-05-01 00:00:00| NaN| NaN| NaN|
+---+----------+------+-------------------+------------------+------------------+------------------+

In [9]:
df3=df1.groupBy(F.col("KEY"), F.window(F.col("Date"), "30 Days").alias("thirty")).agg(F.stddev("metric").alias("last30days_stddev"))

In [10]:
df4=df1.groupBy(F.col("KEY"), F.window(F.col("Date"), "60 Days").alias("sixty")).agg(F.stddev("metric").alias("last60days_stddev"))

In [11]:
df6=df3.join(df4, on=['KEY'])

In [12]:
df5=df1.groupBy(F.col("KEY"), F.window(F.col("Date"), "90 Days").alias("Ninety")).agg(F.stddev("metric").alias("last90days_stddev"))

In [13]:
display(df6.join(df6, on=['KEY']))

KEY,thirty,last30days_stddev,sixty,last60days_stddev,thirty,last30days_stddev,sixty,last60days_stddev
3,"List(2016-04-29T00:00:00.000+0000, 2016-05-29T00:00:00.000+0000)",NaN,"List(2016-04-29T00:00:00.000+0000, 2016-06-28T00:00:00.000+0000)",NaN,"List(2016-04-29T00:00:00.000+0000, 2016-05-29T00:00:00.000+0000)",NaN,"List(2016-04-29T00:00:00.000+0000, 2016-06-28T00:00:00.000+0000)",NaN
2,"List(2016-04-29T00:00:00.000+0000, 2016-05-29T00:00:00.000+0000)",NaN,"List(2016-04-29T00:00:00.000+0000, 2016-06-28T00:00:00.000+0000)",NaN,"List(2016-04-29T00:00:00.000+0000, 2016-05-29T00:00:00.000+0000)",NaN,"List(2016-04-29T00:00:00.000+0000, 2016-06-28T00:00:00.000+0000)",NaN
1,"List(2016-04-29T00:00:00.000+0000, 2016-05-29T00:00:00.000+0000)",4.03732584763727,"List(2016-04-29T00:00:00.000+0000, 2016-06-28T00:00:00.000+0000)",4.03732584763727,"List(2016-04-29T00:00:00.000+0000, 2016-05-29T00:00:00.000+0000)",4.03732584763727,"List(2016-04-29T00:00:00.000+0000, 2016-06-28T00:00:00.000+0000)",4.03732584763727


In [14]:
w=Window().partitionBy("x","y")
w2=Window().partitionBy("x").orderBy(F.col("count").desc())
df.withColumn("count", F.count("y").over(w)).withColumn("Mode", F.first("y").over(w2)).withColumn("y", F.when(F.col("y").isNull(), F.col("Mode")).otherwise(F.col("y"))).drop("count","mode").show()



+---+---+
 x| y|
+---+---+
 1|3.0|
 1|3.0|
 1|9.0|
 1|2.0|
 1|4.0|
 1|5.0|
 1|3.0|
 2|2.0|
 2|2.0|
 2|5.0|
 2|4.0|
 2|3.0|
 2|2.0|
+---+---+

In [15]:
w2=Window().partitionBy("x").orderBy(F.col("count").desc())
df1.withColumn("Mode", F.first("y").over(w2)).withColumn("y", F.when(F.col("y").isNull(), F.col("Mode")).otherwise(F.col("y"))).drop("count","maxc".show()

+---+---+-----+--------+----+
 x| y|count|maxcount|Mode|
+---+---+-----+--------+----+
 1|3.0| 2| 2| 3.0|
 1|3.0| 2| 2| 3.0|
 1|9.0| 1| 1| 3.0|
 1|2.0| 1| 1| 3.0|
 1|4.0| 1| 1| 3.0|
 1|5.0| 1| 1| 3.0|
 1|3.0| 0| 0| 3.0|
 2|2.0| 2| 2| 2.0|
 2|2.0| 2| 2| 2.0|
 2|5.0| 1| 1| 2.0|
 2|4.0| 1| 1| 2.0|
 2|3.0| 1| 1| 2.0|
 2|2.0| 0| 0| 2.0|
+---+---+-----+--------+----+

Out[80]: [Row(x=1, y=3.0), Row(x=2, y=6.0)]

In [17]:
b[2]

Out[76]: 6.0

In [18]:
b

Out[71]: {1: 3.0, 2: 6.0}

In [19]:
df.fillna({'x':3,'x':6},subset=['x','y']).show()

+---+---+
 x| y|
+---+---+
 1|3.0|
 1|3.0|
 1|4.0|
 1|5.0|
 1|NaN|
 2|6.0|
 2|6.0|
 2|7.0|
 2|8.0|
 2|NaN|
+---+---+

In [20]:
df.groupBy('id', 'timestamp', 'amount') \
        .pivot('id') \
        .agg(last('amount', True).over(Window.partitionBy('id').orderBy('timestamp').rowsBetween(Window.unboundedPreceding, Window.unboundedFollowing)).alias('amount'))